In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.callbacks import EarlyStopping


2024-10-30 00:19:19.096550: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-30 00:19:19.108255: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-30 00:19:19.132498: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730236759.156774   73912 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730236759.165063   73912 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 00:19:19.208112: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
landmark_data =np.load("landmark_data.npy")
labels = np.load("labels.npy")

In [3]:
landmark_data = landmark_data / np.max(landmark_data)

# Encode labels as integers and convert to categorical
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

In [4]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(landmark_data, labels_categorical, test_size=0.2, random_state=42)

In [5]:
# Build 1D CNN model
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

/home/diana/.pyenv/versions/3.10.6/envs/sign_language_interpreter/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-30 00:11:14.241416: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

es = EarlyStopping(patience = 5, restore_best_weights=True)

# Train the model
model.fit(X_train[..., np.newaxis], y_train, epochs=40, batch_size=32, validation_data=(X_test[..., np.newaxis], y_test), callbacks=es)
#model.save("asl_sign_language_model.h5")
save_model(model, 'asl_sign_language_model_tf_2.18.keras')

Epoch 1/40
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.4350 - loss: 1.8901 - val_accuracy: 0.9456 - val_loss: 0.2025
Epoch 2/40
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.8631 - loss: 0.4336 - val_accuracy: 0.9695 - val_loss: 0.1150
Epoch 3/40
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.9054 - loss: 0.3007 - val_accuracy: 0.9799 - val_loss: 0.0882
Epoch 4/40
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.9192 - loss: 0.2589 - val_accuracy: 0.9799 - val_loss: 0.0778
Epoch 5/40
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.9306 - loss: 0.2184 - val_accuracy: 0.9841 - val_loss: 0.0745
Epoch 6/40
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.9380 - loss: 0.1964 - val_accuracy: 0.9855 - val_loss: 0.0633
Epoch 7/40
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.9370 - loss: 0.2019 - val_accuracy: 0.9868 - val_loss: 0.0566
Epoch 8/40
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.9479 - los

In [5]:
# Load the trained model

model = tf.keras.models.load_model("asl_sign_language_model_tf_2.18.keras")

2024-10-30 00:19:57.004753: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
def predict_image(directory):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
    mp_drawing = mp.solutions.drawing_utils

    img = cv2.imread(directory)
    img_rbg =  cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img_rbg)

    sequence = []
    sequence_length = 1

    if result.multi_hand_landmarks:
        landmarks = []
        for lm in result.multi_hand_landmarks[0].landmark:
            landmarks.extend([lm.x, lm.y, lm.z])

        # Draw hand landmarks on the frame
        mp_drawing.draw_landmarks(
            img,
            result.multi_hand_landmarks[0],
            mp_hands.HAND_CONNECTIONS
        )

        # Append new frame landmarks to sequence
        sequence.append(landmarks)
        if len(sequence) > sequence_length:
            sequence.pop(0)

        if len(sequence) == sequence_length:
            sequence_input = np.array(sequence).flatten()[np.newaxis, ..., np.newaxis]
            prediction = model.predict(sequence_input)
            predicted_label_index = np.argmax(prediction)
            predicted_label = label_encoder.inverse_transform([predicted_label_index])
            confidence = prediction[0][predicted_label_index]
    return predicted_label


In [7]:
predict_image("/home/diana/code/Koriza274/sign_language_interpreter/basic_models/w.jpg")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1730236804.590832   74447 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730236804.610313   74447 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730236804.635873   74448 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
/home/diana/.pyenv/versions/3.10.6/envs/sign_language_interpreter/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


array(['W'], dtype='<U5')

In [8]:
predict_image("/home/diana/code/Koriza274/sign_language_interpreter/basic_models/a1.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


W0000 00:00:1730236807.919496   74492 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730236807.941919   74492 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/home/diana/.pyenv/versions/3.10.6/envs/sign_language_interpreter/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


array(['A'], dtype='<U5')